In [62]:
import pandas as pd
import numpy as np


## one way ANOVA test
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols


## model
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix 

In [63]:
data = pd.read_csv("withtax.csv")
data = data.rename(columns={"GEO.id2": "id2"})
data.head()

,id2,GEO.id,City,State,"$1 under $25,000","$100,000 under $200,000","$75,000 under $100,000","$200,000 or more","$25,000 under $50,000","$50,000 under $75,000",...,A00200,N01000,A01000,A00101,N18425,A18425,N18450,A18450,N18300,A18300
0,10001,8610000US10001,New York,NY,11690,227126,82593,1057880,58621,82726,...,753264.0,1110,197999.0,1259128,1590,133338,0,0,1610,149260
1,10001,8610000US10001,New York,NY,11690,227126,82593,1057880,58621,82726,...,31491.0,520,-182.0,4688,160,504,130,79,320,1403
2,10001,8610000US10001,New York,NY,11690,227126,82593,1057880,58621,82726,...,75813.0,440,617.0,22026,460,1566,100,84,580,2651
3,10001,8610000US10001,New York,NY,11690,227126,82593,1057880,58621,82726,...,90708.0,440,660.0,54793,780,4315,70,76,860,5696
4,10001,8610000US10001,New York,NY,11690,227126,82593,1057880,58621,82726,...,233340.0,790,4464.0,256806,1750,23359,70,107,1820,27551


In [64]:

subcolumns = ["id2","Establishments not operated for the entire year", "Establishments operated for the entire year", "totalSales", "totalIncome"]
sub_data = data[data.columns[-14:]]
sub_data2 = data[subcolumns]
data2 = sub_data2.merge(sub_data, left_on = sub_data2.index, right_on = sub_data.index)
data2 = data2.drop(['key_0'], axis=1) 
data2.head()

,id2,Establishments not operated for the entire year,Establishments operated for the entire year,totalSales,totalIncome,AGI_STUB,N1,A00100,N00200,A00200,N01000,A01000,A00101,N18425,A18425,N18450,A18450,N18300,A18300
0,10001,66,2225,3.566625e+09,5.331146e+13,6,1650,1275646,1470,753264.0,1110,197999.0,1259128,1590,133338,0,0,1610,149260
1,10001,66,2225,3.566625e+09,5.331146e+13,1,4080,46886,2570,31491.0,520,-182.0,4688,160,504,130,79,320,1403
2,10001,66,2225,3.566625e+09,5.331146e+13,2,2520,93346,2160,75813.0,440,617.0,22026,460,1566,100,84,580,2651
3,10001,66,2225,3.566625e+09,5.331146e+13,3,1790,110732,1580,90708.0,440,660.0,54793,780,4315,70,76,860,5696
4,10001,66,2225,3.566625e+09,5.331146e+13,5,2030,282904,1820,233340.0,790,4464.0,256806,1750,23359,70,107,1820,27551


In [65]:
data2 = data2.rename(columns={"Establishments not operated for the entire year": "estNotEntireYear", "Establishments operated for the entire year":"estEntireYear"}) 
data2.head()

,id2,estNotEntireYear,estEntireYear,totalSales,totalIncome,AGI_STUB,N1,A00100,N00200,A00200,N01000,A01000,A00101,N18425,A18425,N18450,A18450,N18300,A18300
0,10001,66,2225,3.566625e+09,5.331146e+13,6,1650,1275646,1470,753264.0,1110,197999.0,1259128,1590,133338,0,0,1610,149260
1,10001,66,2225,3.566625e+09,5.331146e+13,1,4080,46886,2570,31491.0,520,-182.0,4688,160,504,130,79,320,1403
2,10001,66,2225,3.566625e+09,5.331146e+13,2,2520,93346,2160,75813.0,440,617.0,22026,460,1566,100,84,580,2651
3,10001,66,2225,3.566625e+09,5.331146e+13,3,1790,110732,1580,90708.0,440,660.0,54793,780,4315,70,76,860,5696
4,10001,66,2225,3.566625e+09,5.331146e+13,5,2030,282904,1820,233340.0,790,4464.0,256806,1750,23359,70,107,1820,27551


In [66]:
column_names = list(data2.columns)
for i in range(1,len(column_names)): 
    column_i= column_names[i] 
    formula_str = 'id2' + '~' + column_i
    data_lm = ols(formula_str,data = data2).fit()
    table = sm.stats.anova_lm(data_lm, typ=2)
    print(table)


                        sum_sq        df           F         PR(>F)
estNotEntireYear  3.281498e+11       1.0  546.564826  1.347338e-120
Residual          6.904078e+13  114994.0         NaN            NaN
                     sum_sq        df           F        PR(>F)
estEntireYear  1.719795e+11       1.0  285.801823  4.884909e-64
Residual       6.919695e+13  114994.0         NaN           NaN
                  sum_sq        df           F         PR(>F)
totalSales  2.829639e+11       1.0  470.995109  3.154958e-104
Residual    6.908597e+13  114994.0         NaN            NaN
                   sum_sq        df          F         PR(>F)
totalIncome  4.953609e+11       1.0  827.07394  3.091395e-181
Residual     6.887357e+13  114994.0        NaN            NaN
                sum_sq        df             F  PR(>F)
AGI_STUB  9.023929e-20       1.0  1.495911e-28     1.0
Residual  6.936893e+13  114994.0           NaN     NaN
                sum_sq        df         F         PR(>F)
N1       

In [67]:
## DROP LAST ONE
data2 = data2.drop(columns = ['AGI_STUB','A18425'])
data2.head()


,id2,estNotEntireYear,estEntireYear,totalSales,totalIncome,N1,A00100,N00200,A00200,N01000,A01000,A00101,N18425,N18450,A18450,N18300,A18300
0,10001,66,2225,3.566625e+09,5.331146e+13,1650,1275646,1470,753264.0,1110,197999.0,1259128,1590,0,0,1610,149260
1,10001,66,2225,3.566625e+09,5.331146e+13,4080,46886,2570,31491.0,520,-182.0,4688,160,130,79,320,1403
2,10001,66,2225,3.566625e+09,5.331146e+13,2520,93346,2160,75813.0,440,617.0,22026,460,100,84,580,2651
3,10001,66,2225,3.566625e+09,5.331146e+13,1790,110732,1580,90708.0,440,660.0,54793,780,70,76,860,5696
4,10001,66,2225,3.566625e+09,5.331146e+13,2030,282904,1820,233340.0,790,4464.0,256806,1750,70,107,1820,27551


In [68]:
## 50 percent sampled
## in which 60 percent as train, 20 percent as validate, 20 percent as test
data2 = pd.DataFrame.dropna(data2)
n = int(len(data)*0.5)
data_sampled = data.sample(n)
n = len(data_sampled)
print(n)
train_data = data_sampled.sample(int(n*0.8))
print(len(train_data)) ## 80%
test_data = data_sampled[(data_sampled.index).isin(train_data.index) == False]
print(len(test_data)) ## 20%
validate_data = train_data.sample(int(n*0.2))
print(len(validate_data)) ## 20%
train_data = train_data[(train_data.index).isin(validate_data.index) == False]
print(len(train_data)) ## 60%


57498
45998
11500
11499
34499


In [69]:
############################ model fitting and testing ##########################

In [70]:
x_train = train_data.iloc[:, 4:].to_numpy()
y_train = train_data.iloc[:, 0].to_numpy()
x_test = test_data.iloc[:, 4:].to_numpy()
y_test = test_data.iloc[:, 0].to_numpy()
x_val = validate_data.iloc[:, 4:].to_numpy()
y_val = validate_data.iloc[:, 0].to_numpy()


In [71]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.fit_transform(x_test)
x_val = sc_X.fit_transform(x_val)


In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators = 5)
cross_val_score(clf, x_train, y_train, cv = 3)

/Users/harrisonlee/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


array([0.41583725, 0.6137931 , 0.8099123 ])

In [78]:
print(np.mean(np.array([0.41583725, 0.6137931 , 0.8099123 ])))

0.6131808833333333


In [85]:
rfc = RandomForestClassifier(n_estimators = 5)
rfc.fit(x_train,y_train)
print("acc: ",rfc.score(x_test,y_test))

acc:  0.34730434782608693


In [86]:
for name, importance in zip(column_names[1:], rfc.feature_importances_):
    print(name, "=", importance)


estNotEntireYear = 0.06070041319770342
estEntireYear = 0.05641984477218112
totalSales = 0.058436057488147515
totalIncome = 0.03835106650337015
AGI_STUB = 0.060549783379777275
N1 = 0.06024760783010358
A00100 = 0.044552669338157116
N00200 = 0.03211611841828
A00200 = 0.028122894714418405
N01000 = 0.03966435144605532
A01000 = 0.03415644866164167
A00101 = 0.03661556212483653
N18425 = 0.03938276700731711
A18425 = 0.04160205944293978
N18450 = 0.04900383034082705
A18450 = 0.06107523618296745
N18300 = 0.060404641993691194
A18300 = 0.008792367037457113
